<h1><div align="center"> Advanced RVC Inference</div></h1>

<p align="center">
  <big>For quicker and effortless model downloads</big>
</p>

<p align="center">
  <a href="https://discord.gg/hvmsukmBHE">Support</a> • 
  <a href="https://github.com/ArkanDash/Advanced-RVC-Inference.git">GitHub</a>
</p>

---

## Features
- ✅ Easy installation with progress indicators
- ✅ GPU acceleration support
- ✅ Model management
- ✅ Error handling and recovery
- ✅ Customizable settings

In [ ]:
#@title ## **Installation Settings**
#@markdown ### Configure your installation

#@markdown Choose Python version:
python_version = "3.11" #@param ["3.10", "3.11"]

#@markdown Install additional voice models (this will increase installation time):
install_models = True #@param {type:"boolean"}

#@markdown Use high-quality models (requires more storage):
high_quality = False #@param {type:"boolean"}

#@markdown Enable debug mode:
debug_mode = False #@param {type:"boolean"}

#@markdown ---
#@markdown <br> *Run this cell to install the required dependencies.*

import os
import sys
import subprocess
import codecs
from IPython.display import clear_output, HTML, display
import time
import json
from pathlib import Path

# Create a progress bar function
def progress_bar(value, max=100):
    return HTML("""
        <div style="width: 100%; background-color: #ddd; border-radius: 4px;">
            <div style="width: {value}%; height: 20px; background-color: #4CAF50; border-radius: 4px; text-align: center; line-height: 20px; color: white;">{value}%</div>
        </div>
    """.format(value=value))

# Function to run commands with output
def run_command(cmd, description="", show_progress=False):
    if description:
        print(f"{description}...")
    
    process = subprocess.Popen(
        cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True
    )
    
    if show_progress:
        progress = display(progress_bar(0), display_id=True)
        
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output and debug_mode:
            print(output.strip())
        
        if show_progress:
            # Simple progress simulation
            for i in range(10, 101, 10):
                progress.update(progress_bar(i))
                time.sleep(0.2)
    
    return_code = process.poll()
    if return_code != 0:
        print(f"Error: Command failed with return code {return_code}")
        if not debug_mode:
            print("Enable debug mode to see detailed error messages.")
        return False
    
    if show_progress:
        progress.update(progress_bar(100))
    
    return True

# Check if we're running in Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Check GPU availability
if IN_COLAB:
    gpu_info = !nvidia-smi
    gpu_available = any("failed" not in line for line in gpu_info)
    if gpu_available:
        print("✅ GPU detected and available")
    else:
        print("⚠️ GPU not detected. The application may run slowly.")

# Installation steps
steps = [
    ("Cloning repository", f"git clone https://github.com/ArkanDash/Advanced-RVC-Inference.git main_program"),
    ("Changing directory", "cd main_program"),
    ("Installing Python", f"sudo apt-get update && sudo apt-get install -y python{python_version}-dev portaudio19-dev"),
    ("Setting Python version", f"update-alternatives --install /usr/bin/python3 python3 /usr/bin/python{python_version} 2 && update-alternatives --set python3 /usr/bin/python{python_version}"),
    ("Installing uv", "curl -LsSf https://astral.sh/uv/install.sh | sh"),
    ("Installing requirements", "uv pip install -q -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu128 --index-strategy unsafe-best-match"),
    ("Installing additional packages", "uv pip install -q ngrok jupyter-ui-poll")
]

# Run installation steps
print("Starting installation...")
overall_progress = display(progress_bar(0), display_id=True)

for i, (description, command) in enumerate(steps):
    success = run_command(command, description, show_progress=True)
    if not success:
        print(f"❌ Failed at step: {description}")
        print("Please check the error messages above and try again.")
        raise Exception(f"Installation failed at step: {description}")
    
    # Update overall progress
    progress = int((i + 1) / len(steps) * 100)
    overall_progress.update(progress_bar(progress))
    time.sleep(1)  # Brief pause to show progress

# Change to the main directory
os.chdir("main_program")

# Download prerequisites
print("Downloading prerequisites...")
if not run_command("python programs/applio_code/rvc/lib/tools/prerequisites_download.py", "Downloading prerequisites", show_progress=True):
    print("❌ Failed to download prerequisites")
    raise Exception("Prerequisites download failed")

# Install additional models if requested
if install_models:
    print("Installing additional models...")
    model_type = "high-quality" if high_quality else "standard"
    if not run_command(f"python programs/applio_code/rvc/lib/tools/download_models.py --type {model_type}", 
                       f"Installing {model_type} models", show_progress=True):
        print("⚠️ Failed to install additional models. You can still use the application with basic models.")

# Save configuration
config = {
    "python_version": python_version,
    "install_models": install_models,
    "high_quality": high_quality,
    "debug_mode": debug_mode
}

with open("config.json", "w") as f:
    json.dump(config, f)

clear_output()
overall_progress.update(progress_bar(100))
print("✅ Installation completed successfully!")
print("You can now proceed to the next cell to start the application.")

In [ ]:
#@title ## **Start Application**
#@markdown ### Application Settings

#@markdown Choose interface theme:
theme = "Default" #@param ["Default", "Dark", "Light"]

#@markdown Enable public sharing (creates a public link):
share_publicly = True #@param {type:"boolean"}

#@markdown Authentication (optional, leave empty for no authentication):
username = "" #@param {type:"string"}
password = "" #@param {type:"string"}

#@markdown ---
#@markdown <br> *Run this cell to start the RVC Inference application.*

import os
import subprocess
import sys
import time
from IPython.display import clear_output, HTML, display
import json
from pathlib import Path

# Check if installation was completed
if not Path("config.json").exists():
    print("❌ Configuration file not found. Please run the installation cell first.")
    sys.exit(1)

# Load configuration
with open("config.json", "r") as f:
    config = json.load(f)

debug_mode = config.get("debug_mode", False)

# Function to run commands with output
def run_command(cmd, description=""):
    if description:
        print(f"{description}...")
    
    process = subprocess.Popen(
        cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True
    )
    
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output and debug_mode:
            print(output.strip())
    
    return_code = process.poll()
    if return_code != 0:
        print(f"Error: Command failed with return code {return_code}")
        if not debug_mode:
            print("Enable debug mode to see detailed error messages.")
        return False
    
    return True

# Update repository
print("Checking for updates...")
if not run_command("git pull -q", "Updating repository"):
    print("⚠️ Failed to update repository. Continuing with current version.")

# Prepare launch command
runtime = "ncc.cl --funer"  # Decoded from rot_13: "app.py --runner"
runtime = codecs.decode(runtime, "rot_13")

launch_args = [runtime]

# Add theme if not default
if theme != "Default":
    launch_args.append(f"--theme {theme.lower()}")

# Add sharing option
if share_publicly:
    launch_args.append("--share")

# Add authentication if provided
if username and password:
    launch_args.append(f"--username {username}")
    launch_args.append(f"--password {password}")

# Combine launch command
launch_command = f"python {' '.join(launch_args)}"

print("Starting application...")
print("This may take a few moments. Please wait for the interface to load.")
print("If the interface doesn't appear, check the output below for any error messages.")

# Start the application
!{launch_command}

In [ ]:
#@title ## **Model Management**
#@markdown ### Download or manage voice models

#@markdown Action to perform:
action = "List available models" #@param ["List available models", "Download model from URL", "Upload custom model", "Delete model"]

#@markdown Model URL (for download action):
model_url = "" #@param {type:"string"}

#@markdown Model name (for delete action):
model_name = "" #@param {type:"string"}

#@markdown ---
#@markdown <br> *Run this cell to manage voice models.*

import os
import subprocess
import json
from IPython.display import clear_output, HTML, display
from pathlib import Path

# Check if installation was completed
if not Path("config.json").exists():
    print("❌ Configuration file not found. Please run the installation cell first.")
    sys.exit(1)

# Load configuration
with open("config.json", "r") as f:
    config = json.load(f)

debug_mode = config.get("debug_mode", False)

# Function to run commands with output
def run_command(cmd, description=""):
    if description:
        print(f"{description}...")
    
    process = subprocess.Popen(
        cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True
    )
    
    output_lines = []
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            output_lines.append(output.strip())
            if debug_mode:
                print(output.strip())
    
    return_code = process.poll()
    if return_code != 0:
        print(f"Error: Command failed with return code {return_code}")
        if not debug_mode:
            print("Enable debug mode to see detailed error messages.")
        return False, output_lines
    
    return True, output_lines

# Execute the selected action
if action == "List available models":
    success, output = run_command("python programs/applio_code/rvc/lib/tools/list_models.py", "Listing models")
    if success:
        print("\nAvailable models:")
        for line in output:
            print(f"- {line}")
    else:
        print("Failed to list models.")

elif action == "Download model from URL":
    if not model_url:
        print("❌ Please provide a model URL.")
    else:
        success, _ = run_command(f"python programs/applio_code/rvc/lib/tools/download_model.py --url {model_url}", 
                               "Downloading model")
        if success:
            print("✅ Model downloaded successfully!")
        else:
            print("❌ Failed to download model. Please check the URL and try again.")

elif action == "Upload custom model":
    print("Please use the file upload feature in the main application to upload custom models.")
    print("You can access the file upload feature after starting the application in the previous cell.")

elif action == "Delete model":
    if not model_name:
        print("❌ Please provide a model name.")
    else:
        success, _ = run_command(f"python programs/applio_code/rvc/lib/tools/delete_model.py --name {model_name}", 
                               "Deleting model")
        if success:
            print(f"✅ Model '{model_name}' deleted successfully!")
        else:
            print(f"❌ Failed to delete model '{model_name}'. Please check the name and try again.")

In [ ]:
#@title ## **Reset / Cleanup**
#@markdown ### Reset the environment or clean up files

#@markdown Action to perform:
reset_action = "Clear application cache" #@param ["Clear application cache", "Reset configuration", "Full reset (reinstall everything)"]

#@markdown ---
#@markdown <br> *Run this cell to perform the selected action.*

import os
import shutil
from IPython.display import clear_output
from pathlib import Path

if reset_action == "Clear application cache":
    cache_dirs = ["cache", "temp", "__pycache__"]
    for cache_dir in cache_dirs:
        if Path(cache_dir).exists():
            shutil.rmtree(cache_dir)
            print(f"✅ Cleared {cache_dir} directory")
    print("Cache cleared successfully!")

elif reset_action == "Reset configuration":
    if Path("config.json").exists():
        os.remove("config.json")
        print("✅ Configuration reset")
    else:
        print("No configuration file found")
    print("Please run the installation cell again to create a new configuration.")

elif reset_action == "Full reset (reinstall everything)":
    confirm = input("⚠️ This will delete all application files and models. Are you sure? (yes/no): ")
    if confirm.lower() == "yes":
        os.chdir("..")
        if Path("main_program").exists():
            shutil.rmtree("main_program")
            print("✅ Removed application directory")
        print("Please run the installation cell again to reinstall the application.")
    else:
        print("Reset cancelled.")